# Prophet Single Model

## Data Description

Input file: Historical Product Demand.csv

Description: CSV data file containing product demand for encoded product id's

Size of Data: (1048575, 5)

Features: Product_Code, Warehouse, Product_Category, Date, Order_Demand

Period: 2012-01-01 ~ 2017-01-09


---

In [24]:
# DataFrame
import pandas as pd
import numpy as np
import random
import time
from datetime import datetime, date

# Preprocessing
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import itertools

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pandas.errors import SettingWithCopyWarning
#warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.filterwarnings("ignore") # 경고 무시

# Save the log
import os

# Prophet 
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.plot import add_changepoints_to_plot
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.model_selection import RandomizedSearchCV

# Metric
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error

## Data Explore

In [25]:
# Data Loading
df = pd.read_csv('HPD_0416.csv')
# convert the string to the datetype
df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
df

,Date,Product_Code,Product_Category,Order_Demand
0,2012-01-05,Product_0025,Category_005,1600.0
1,2012-01-06,Product_0025,Category_005,1000.0
2,2012-01-07,Product_0025,Category_005,0.0
3,2012-01-08,Product_0025,Category_005,0.0
4,2012-01-09,Product_0025,Category_005,0.0
...,...,...,...,...
14551,2016-12-23,Product_2004,Category_005,0.0
14552,2016-12-24,Product_2004,Category_005,0.0
14553,2016-12-25,Product_2004,Category_005,0.0
14554,2016-12-26,Product_2004,Category_005,2000.0


In [26]:
print(df.info())
print('-------------------------')
print("")
print("The Number of unique")
print('-------------------------')
print('Product code:\t', df.Product_Code.nunique())
print('Category:\t', df.Product_Category.nunique())
print('-------------------------')
print("The Product Code:")
print("")
for i, code in enumerate(df['Product_Code'].unique()):
    print(i+1, code)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14556 entries, 0 to 14555
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              14556 non-null  datetime64[ns]
 1   Product_Code      14556 non-null  object        
 2   Product_Category  14556 non-null  object        
 3   Order_Demand      14556 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 455.0+ KB
None
-------------------------

The Number of unique
-------------------------
Product code:	 8
Category:	 5
-------------------------
The Product Code:

1 Product_0025
2 Product_0739
3 Product_0901
4 Product_1154
5 Product_1248
6 Product_1295
7 Product_1378
8 Product_2004


---

## Split the data
- Input
     data: dataframe with dates and Demand data
     
- output
    - train:  2012-01-01 ~ 2015-06-30 
    - Valid:  2015-07-01 ~ 2015-12-31
    - test :  2016-01-01 ~ 2017-01-06 
    
     
- time_steps: # of the input time steps 
- for_periods: # of the output time steps 

In [27]:
# train과 test로 데이터 split
def split_data(df):
    df['ds'] = pd.to_datetime(df['ds']).copy()
    
    train_df = df[(df['ds'] <'2015-07-01')].sort_values('ds', ascending=True)
    valid_df = df[(df['ds'] >= '2015-07-01') & (df['ds'] < '2016-01-01')].sort_values('ds', ascending=True)
    test_df = df[(df['ds'] >= '2016-01-01')].sort_values('ds', ascending=True) 
    
    return train_df, valid_df, test_df

### 파라미터 최적화
    - Random Search

In [28]:
def optimize_prophet(product_df, trials):
    # Prophet' parameters
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
        'seasonality_mode': ['additive', 'multiplicative'],
        'changepoint_range': [0.8, 0.9, 1.0],
    }
    
    # 파라미터 조합 생성
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  
    cutoff = pd.to_datetime(['2015-06-30'])
    valid_cut = pd.to_datetime(['2015-12-31'])
    prediction_periods = (valid_cut - cutoff).days
    
    # 랜덤 샘플링으로 50개의 조합 선택
    selected_params = random.sample(all_params, trials)
    
    # 파리미터 평가를 위해  validation_data로 cv진행
    for params in selected_params:
        m = Prophet(**params)
        m.add_country_holidays(country_name = 'US') # 미국 공휴일 추가
        m.fit(product_df)
        df_cv = cross_validation(m, cutoffs=cutoff, horizon=str(prediction_periods[0])+' days')
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])
        
    best_params = selected_params[np.argmin(rmses)]
    return best_params

## Prophet

In [29]:
def Prophet_single(product_df, optimize_trials=30): # optimize_trials: Search 횟수

    train_df, valid_df, test_df = split_data(product_df)
    # 파라미터 최적화
    best_params = optimize_prophet(product_df, optimize_trials)
    
    # 최적 파라미터를 사용한 모델 훈련
    best_model = Prophet(**best_params)
    best_model.add_country_holidays(country_name='US')
    best_model.fit(train_df)
    
    # Prophet으로 예측
    periods=(test_df['ds'].max() - train_df['ds'].max()).days
    future = best_model.make_future_dataframe(periods=periods) 
    forecast = best_model.predict(future)
    
    #예측결과 저장
    res_df = pd.merge(test_df, forecast[['ds','yhat']], on='ds')
    res_df = res_df[['ds', 'y', 'yhat']]
    res_df.set_index('ds')
    res_df.rename(columns={'yhat': 'Pred'}, inplace=True)
    
    # 예측값이 음수인 것은 0으로 대치
    res_df['Pred'] = res_df['Pred'].apply(lambda x: 0 if x < 0 else x)
    scaler = MinMaxScaler()
    # 'y'와 'yhat' 열을 정규화
    res_df[['y_norm', 'Pred_norm']] = scaler.fit_transform(res_df[['y', 'Pred']])
    res_df.set_index('ds', inplace=True)
        
        # 모델과 예측값 딕셔너리 반환
    return best_model, res_df

## Plot the result

In [30]:
def actual_pred_plot(product_code, res_df, metric_df, normalize):
    today = date.today()
    """
    Plot the actual vs predition and save the figure in the given directory
    """
    
    save_path = os.path.join("Result", "Single_Prophet_Result", product_code)
    save_name = f'{product_code}_all_result'
    
    title = f"Pred Actual Plot - {product_code}"
    actual = res_df['y']
    pred = res_df['Pred']
    
    if normalize: 
        title += "(Normalized)"
        actual = res_df['y_norm']
        pred = res_df['Pred_norm']
        save_name += "_normalized"
    # Plot   
    plt.figure(figsize=(16, 8))
    plt.title(title, fontsize=20)
    plt.xlabel("Time", fontsize=14)
    plt.ylabel("Order Demand", fontsize=14)
    plt.plot(actual, label ='Actual', alpha=0.6)
    plt.plot(pred, label='Prediction', alpha=0.8)
    plt.legend(loc="upper right")
        
    # Plot 결과 저장
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    # save the figure
    today_date = f'_{today.month:02d}{today.day:02d}'
    plt.savefig(os.path.join(save_path, save_name+'.png'))
    metric_df.to_csv(os.path.join(save_path, save_name+'.csv'))
        
    plt.close('all') # close all figures to free up memory

## Metrics

In [31]:
# Model Metric
def mase(training_series, testing_series, prediction_series):
    n = training_series.shape[0]
    d = np.abs(np.diff(training_series)).sum() / (n-1)
    
    errors = np.abs(testing_series - prediction_series)
    return errors.mean() / d

# Model Metric
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / (actual+1)))

def nrmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, squared=False)
    target_mean = np.mean(y_true)
    nrmse = mse / target_mean
    return nrmse

In [32]:
def calculate_metrics(product_code, res_df, normalize):
    # 정규화 옵션이 True인 경우 정규화된 데이터 사용, 그렇지 않으면 원래 데이터 사용
    if normalize:
        actual = res_df['y_norm']
        pred = res_df['Pred_norm']
    else:
        actual = res_df['y']
        pred = res_df['Pred']

    # 메트릭 계산
    # MASE = mase(np.array(train_series), np.array(actual), pred) 
    MAPE = mape(actual, pred) 
    RMSE = mean_squared_error(actual, pred)**0.5 
    MAE = mean_absolute_error(actual,pred) 
    NRMSE = nrmse(actual,pred) 
    # RMSLE = mean_squared_log_error(actual, pred)**0.5 

    # 계산된 메트릭을 데이터프레임에 추가
    metric_df = pd.DataFrame({'MAPE':[round(MAPE, 4)],
                           'RMSE':[round(RMSE, 4)],
                           'MAE':[round(MAE, 4)],
                           'NRMSE':[round(NRMSE, 4)]},
                            index= [product_code])

    return metric_df

---

## Check the Result

### Total Result

In [33]:
def make_all_result_df(res_df):
    
    res_df.loc[res_df['Pred']<0, 'Pred']=0 # 음수 예측 값은 0으로 대치
    
    # 날짜(Date) 열은 정규화하지 않으므로 제외
    result_norm = res_df[['Pred', 'y']]
    
    # MinMaxScaler를 이용하여 정규화합니다.
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(result_norm)
    
    # 정규화된 데이터를 데이터 프레임에 반영합니다.
    res_df['Pred_norm'] = normalized_data[:,0]
    res_df['y_norm'] = normalized_data[:,1]
    return res_df

In [34]:
def execute_single_Prophet(product_code, optimize_trials=50): # 최적화 횟수
    start_time = time.time()
    
    product_code = product_code # 예측하고자 하는 코드 입력
    product_df = df[df['Product_Code']== product_code].reset_index(drop=True)
    product_df = product_df[['Date', 'Order_Demand']]
    product_df.rename(columns={'Date': 'ds', 'Order_Demand': 'y'}, inplace=True)
    # Prophet 단일 모델
    best_model, res_df = Prophet_single(product_df, optimize_trials) 
    res_df = make_all_result_df(res_df)
    
    # 모델 Metric과 Pred_Actual Plot 저장
    metric_df_norm = calculate_metrics(product_code, res_df, True)
    metric_df= calculate_metrics(product_code, res_df, False)
    
    actual_pred_plot(product_code, res_df, metric_df_norm, True)
    actual_pred_plot(product_code, res_df, metric_df, False)
    
    # 실행시간 확인
    elapsed_time_seconds = time.time() - start_time
    elapsed_time_minutes = elapsed_time_seconds / 60
    print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))
    return metric_df

---

## Whole Process
    - product_code에 str으로 예측하고자 하는 코드를 입력
    - ['Product_0025', 'Product_0739', 'Product_0901', 'Product_1154',
       'Product_1248', 'Product_1295', 'Product_1378', 'Product_2004']

In [35]:
for code in ['Product_0025', 'Product_0739', 'Product_0901', 'Product_1154','Product_1248', 'Product_1295', 'Product_1378', 'Product_2004']:
    print("==================================")
    print(f"========== { code } ==========")
    print("==================================")
    execute_single_Prophet(code)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


========== Product_0025 ==========


  0%|          | 0/1 [00:00<?, ?it/s]

INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


  0%|          | 0/1 [00:00<?, ?it/s]

INFO:prophet:Skipping MAPE because y close to 0
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


실행 시간: 2.31 분
